In [149]:
import duckdb
con = duckdb.connect(database=':memory:')

# 1699. Number of Calls Between Two Persons

In [2]:
con.execute('''
CREATE TABLE IF NOT EXISTS Calls (
    from_id int NOT NULL,
    to_id int NOT NULL,
    duration int NOT NULL
    )
''')

con.execute('''
INSERT INTO Calls (from_id,  to_id, duration)
VALUES
(1  ,2     , 59   ),
(2  ,1     , 11   ),
(1  ,3     , 20   ),
(3  ,4     , 100  ),
(3  ,4     , 200  ),
(3  ,4     , 200  ),
(4  ,3     , 499  ),; ''')

In [3]:
con.execute("""
SELECT person1, person2, sum(calls) AS call_count,  sum(duration) as total_duration
FROM 
( select 
    from_id as person1, 
    to_id as person2, 
    1 as calls, 
    duration
    from Calls
    where from_id < to_id
    union all
    select 
    to_id as person1, 
    from_id as person2, 
    1 as calls, 
    duration
    from Calls
    where to_id < from_id)  c
GROUP BY person1, person2
""").fetchdf()

,person1,person2,call_count,total_duration
0,1,2,2.0,70.0
1,1,3,1.0,20.0
2,3,4,4.0,999.0


# 1445. Apples & Oranges

In [22]:
con.execute('''
CREATE TABLE IF NOT EXISTS Sales (
    sale_date date NOT NULL,
    fruit varchar(32) NOT NULL,
    sold_num int NOT NULL
    )
''')

con.execute('''
INSERT INTO Sales (sale_date,  fruit, sold_num)
VALUES
('2020-05-01'   , 'apples'    , 10   ),
('2020-05-01 ', 'oranges', 8),
(' 2020-05-02  ', 'apples', 15),
('2020-05-02 ', 'oranges', 15),


('2020-05-03 ', 'apples', 20),
('2020-05-03 ', 'oranges', 0),
('2020-05-04 ', 'apples', 15),
('2020-05-04 ', 'oranges', 16)


; ''')

In [78]:
con.execute("""
SELECT sale_date, sum(sold_num) as diff
FROM (
SELECT  sale_date, count(fruit) as fruit, sum(sold_num) as sold_num
FROM Sales
WHERE fruit = 'apples'
GROUP BY sale_date

union 
SELECT  sale_date, count(fruit) as fruit, -sum(sold_num) as sold_num
FROM Sales
WHERE fruit = 'oranges'
GROUP BY sale_date) c

GROUP BY sale_date

""").fetchdf()

,sale_date,diff
0,2020-05-01,2.0
1,2020-05-02,0.0
2,2020-05-03,20.0
3,2020-05-04,-1.0


# 1251. Average Selling Price

In [89]:
con.execute('''
CREATE TABLE IF NOT EXISTS Prices (
    product_id int NOT NULL,
    start_date date NOT NULL,
    end_date date NOT NULL,
    price int NOT NULL
    )
''')

con.execute('''
CREATE TABLE IF NOT EXISTS UnitsSold (
    product_id int NOT NULL,
    purchase_date date NOT NULL,
    units int NOT NULL
    )
''')

con.execute('''
INSERT INTO Prices (product_id,  start_date, end_date, price)
VALUES
(1,  '2019-02-17', '2019-02-28',  5    ),
(1,  '2019-03-01', '2019-03-22',  20    ),
(2,  '2019-02-01', '2019-02-20',  15   ),
(2,  '2019-02-21', '2019-03-31',  30    )

; ''')


con.execute('''
INSERT INTO UnitsSold (product_id,  purchase_date, units)
VALUES
(1, '2019-02-25',  100    ),
(1,  '2019-03-01',  15    ),
(2,  '2019-02-10',  200  ),
(2,   '2019-03-22',  30    )

; ''')

In [129]:
con.execute("""
SELECT product_id, ROUND(sum(income) / sum(units),2) as 'average_price'
FROM(
SELECT  UnitsSold.product_id, (UnitsSold.units * Prices.price) AS 'income' , units
FROM Prices
JOIN UnitsSold ON Prices.product_id = UnitsSold.product_id WHERE purchase_date <= end_date and purchase_date>=start_date) c
GROUP BY product_id

""").fetchdf()

,product_id,average_price
0,1,6.0
1,2,16.0


# 1571. Warehouse Manager

In [150]:
con.execute('''
CREATE TABLE IF NOT EXISTS Warehouse (
    name varchar NOT NULL,
    product_id int NOT NULL,
    units int NOT NULL
    )
''')

con.execute('''
CREATE TABLE IF NOT EXISTS Products (
    product_id int NOT NULL,
    product_name varchar NOT NULL,
    Width int NOT NULL,
    Length int NOT NULL,
    Height int NOT NULL
    )
''')

con.execute('''
INSERT INTO Warehouse (name,  product_id, units)
VALUES
('LCHouse1', 1,1),
('LCHouse1', 2,10),
('LCHouse1', 3,5),
('LCHouse2', 1,2),
('LCHouse2', 2,2),
('LCHouse3', 4,1)




; ''')


con.execute('''
INSERT INTO Products (product_id,  product_name, Width,Length, Height )
VALUES
(1, 'LC-TV', 5,50,40),
(2, 'LC-KeyChain', 5,5,5),
(3, 'LC-Phone', 2,10,10),
(4, 'LC-T-Shirt', 4,10,20)

; ''')

In [169]:
con.execute("""
SELECT name as warehouse_name, sum(Width*Length*Height* units) as volume
FROM Warehouse 
JOIN Products on Products.product_id = Warehouse.product_id
GROUP BY name

""").fetchdf()

,warehouse_name,volume
0,LCHouse1,12250.0
1,LCHouse2,20250.0
2,LCHouse3,800.0
